This notebook contains code examples to connect to AWS S3, Redshift, MySQL, Hive, and SQL Server

### S3

In [ ]:
!pip install pandas==0.19 # for boto compatibility
!pip install boto3
import boto3
def get_s3_client(access_key, secret_key):
    
    """
    
    Initialize s3 connection client. Run before any of the other s3 utility functions
    
    """
    s3_client = boto3.client(service_name = 's3', 
                      aws_access_key_id = access_key, 
                      aws_secret_access_key = secret_key)
    return(s3_client)

def s3_ls(s3_client, s3_bucket, path):
    
    """
    
    List contents of s3 directory specified in 'path'
    
    """
    
    if path != '' and path[-1] != '/':
        path += '/'
        
    files = []
    directories = []
    
    try:
        for fname in s3_client.list_objects(Bucket = s3_bucket, Prefix = path)['Contents']:
            
            if '/' not in fname['Key'].replace(path,''):
                files.append(fname['Key'].replace(path,''))
            elif fname['Key'].replace(path,'').split('/')[0] + '/' not in directories:
                directories.append(fname['Key'].replace(path,'').split('/')[0] + '/')
                
    except KeyError:
        return('Directory Not Found')

    return(directories + files)

def s3_pull_file(s3_client, s3_bucket, s3_key, local_path = None):
    """
    
    Pull a file (any format) from S3 into the platform environment
    Please supply the local_path without the file name (i.e. 'testing/test' NOT 'testing/test/test.csv')
    
    After the file has been pulled in, 
    it can be read into Python using the usual methods (e.g. open())
    
    If the local_path argument is not specified, the file will be placed in /home/jupyter/
    
    """
    # Create the directory to store the file on the platform
    filename = s3_key.split('/')[-1]
    if local_path is None:
        local_dir = '/home/jupyter/'.format(filename)
    else:
        local_dir = '/home/jupyter/{0}'.format(local_path)
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)
        
    # Download the file
    s3_client.download_file(Bucket = s3_bucket, Key = s3_key, Filename = local_dir + '/' + filename)
    
    print("Your file is now available at '{0}'".format(local_dir))
    
def s3_push_file(s3_client, s3_bucket, local_filepath, s3_filepath):
    """
    
    Push a file from the platform environment into S3
    
    """
    try:
        s3_client.upload_file(local_filepath, s3_bucket, s3_filepath)
        print("Uploaded to " + s3_filepath)
    except BaseException as e:
        print("Upload error for " + local_filepath)
        print(str(e))
        
# Example calls
s3_bucket = 'example_bucket'

s3_client = get_s3_client(access_key = 'example_access_key'
                           , secret_key = 'example_secret_key')

s3_ls(s3_client = s3_client
      , s3_bucket = s3_bucket
      , path = 'example/key')

s3_pull_file(s3_client = s3_client
             , s3_bucket = s3_bucket
             , s3_key = 'example/key/my.csv'
             , local_path = 'local_example/test')

s3_push_file(s3_client = s3_client
             , s3_bucket = s3_bucket
             , local_filepath = 'example/path.txt'
             , s3_filepath = 'example/key')

# Saves a pandas dataframe as a csv in S3
def s3_DFtoCSV(dataframe, file_name):
    """
    
    Saves a pandas dataframe as a csv
    
    """   
    
    s3, bucket = get_s3_client()
    
    str_csv=dataframe.to_csv(None, sep='\t',index=False)
    s3.put_object(Body=str_csv, Bucket=bucket, Key=file_name) 

# Loads a CSV into a pandas dataframe
def s3_CSVtoDF(file_name, **kwargs):
    
    """
    
    Stream a data file from S3 into a dataframe
    
    All **kwargs are passed to pandas.read_csv() and must
    therefore be valid keyword arguments of that function
    
    """
    
    s3, bucket = get_s3_client()
    obj = s3.get_object(Bucket=bucket, Key=file_name)
    
    return pd.read_csv(obj['Body'], **kwargs)

### Redshift

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    database = 'example_dbname',
    port = 'example_port',
    password = 'example_pw',
    user = 'example_user',
    host = 'example_host'
)

def results(query):
    return pd.read_sql(query, conn)

example_query = '''
example query;
'''

data = results(example_query)

### MySQL

In [ ]:
!sudo apt-get -y update  # Can run this in a terminal 
!sudo apt-get -y install libmysqlclient-dev # Can run this in a terminal or include in requirements

!pip install MySQL-python # can include in requirements file
!pip install sqlalchemy # can include in requirements file

from sqlalchemy import create_engine
import MySQLdb as mdb

conn = mdb.connect(os.environ['HOSTSQL'], os.environ['USERSQL'], os.environ['PASSWDSQL'], os.environ['DBSQL'])
cur = conn.cursor()
cur.execute('''
example_statement;
'''.format(dbsql, table_name))

# To append data from a pandas dataframe onto an existing table
engine = create_engine('mysql+mysqldb://{0}:{1}@{2}/{3}'.format(os.environ['USERSQL'],  os.environ['PASSWDSQL'], os.environ['HOSTSQL'], os.environ['DBSQL']), echo = False)
result.to_sql(name = '{}'.format(table_name), con = engine, if_exists = 'append', index=False)

### Hive

In [ ]:
import jaydebeapi
from os import listdir
from os.path import isfile, join
import pandas as pd
import jpype

jvmPath = jpype.getDefaultJVMPath()
jpype.startJVM(jvmPath,  "-Djava.security.auth.login.config=/home/jupyter/jaas.conf", "-Djava.security.krb5.conf=/etc/krb5.conf", "-Djavax.security.auth.useSubjectCredsOnly=false", "-Djava.ext.dirs=/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/ext:/usr/lib/hive/lib:/usr/lib/hadoop:/usr/lib/hadoop/lib", "-Dlog4j.configuration=file:///etc/hive/conf/hive-log4j.properties", "-Dsun.security.krb5.debug=true")
conn = jaydebeapi.connect('org.apache.hive.jdbc.HiveDriver', 'example_address', ["",""])
df1 = pd.read_sql("example_query", conn)
print(df1)
conn.close()

### SQL Server

In [ ]:
import pymssql

server = getenv("PYMSSQL_TEST_SERVER")
user = getenv("PYMSSQL_TEST_USERNAME")
password = getenv("PYMSSQL_TEST_PASSWORD")

conn = pymssql.connect(server, user, password, "tempdb")